In [ ]:
# conda activate dash; pip install holoviews -U; cd Interactive-Dashboard ; panel serve Visualisation.ipynb
# git add Visualisation.ipynb

import holoviews as hv
import hvplot.pandas
import panel as pn
import pandas as pd
import requests

from decouple import config
from datetime import datetime

pn.extension('tabulator')

df = pd.read_csv('Database.csv')
idf = df.interactive()

class MainGraphs:

    def __init__(self):
        self.goal_lines = [(hv.HLine(100)).opts(color='pink', line_dash='dashed'), (hv.HLine(8)).opts(color='pink', line_dash='dashed'), (hv.HLine(10000)).opts(color='pink', line_dash='dashed'), 
        (hv.HLine(2)).opts(color='pink', line_dash='dashed'), (hv.HLine(550)).opts(color='pink', line_dash='dashed')]
        self.full_selections = pn.widgets.Select(groups={'Health': ['Steps', 'Steps Score', 'Active Cals',
        'Active Cals Score', 'Diet'], 'Productivity': ['Work Hours', 'Work Score', 'Reading Hours', 'Reading Score'],
        'Personal': ['Cleaning', 'Dogs', 'Self-Care']})
        self.category_selections = pn.widgets.RadioButtonGroup(options=['Overall', 'Health', 'Productivity', 'Personal'])
        self.datapoints = len(df)
    
    def make_category_graph(self):
        days_since_slider = pn.widgets.IntSlider(name='Days Since Today',
        start = 1, end = self.datapoints, step = 1, width=200, value = 7)
        category_data = idf[days_since_slider>=idf.Days][self.category_selections]
        category_graph = ((category_data.hvplot(x='Days Since Today', y=self.category_selections, color='hotpink', kind='step', ylim=(50,120), width=600))) *(self.goal_lines)[0]
        return category_graph

    def make_detailed_graph(self):
        days_since_slider = pn.widgets.IntSlider(name='Days Since Today',
        start = 1, end = self.datapoints, step = 1, width=200, value = 7)
        detailed_data = idf[days_since_slider>=idf.Days][self.full_selections]
        detailed_graph =  (detailed_data.hvplot(x='Days Since Today', y=self.full_selections, color='hotpink', kind='step', width =400))
        for i in self.goal_lines:
            detailed_graph = detailed_graph*i
        return detailed_graph

    def make_heatmap(self):
        heatmap = idf.hvplot(x='Month', y='Day of Month', C='Overall', kind='heatmap',
                              cmap=[ '#FFB7C5', '#FFA6C9','#FC8EAC' ,'#FF69B4', '#E75480', '#A94064'], width = 600)
        return heatmap

    def make_box_plot(self):
        boxplot = (idf.hvplot(y=self.category_selections, by='Month', width = 400, color='hotpink', outlier_color='white', whisker_line_color='white', kind ='box'))*(self.goal_lines)[0]
        return boxplot

class SideBar:

    def get_quote(self):
        url = 'https://stoicquotesapi.com/v1/api/quotes/random'
        response = requests.get(url)
        quote = (response.json()).get('body')
        quote_text = pn.widgets.StaticText(value=quote, width=200, style={'color':'white', 'font-family': "monospace"})
        return quote_text   

    def nasa_image_of_the_day(self):
        url = 'https://api.nasa.gov/planetary/apod?api_key=PZcnX4xvaDZt6n394qdhjTT9p9Jvwex3oTqMofpt'
        response = requests.get(url)
        nasa_dict = (response.json())['hdurl']
        image_of_the_day = pn.pane.JPG(nasa_dict, width=200)
        return image_of_the_day

    def get_manutd_next_game_data(self):
        url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"
        querystring = {"team":"33","next":"1"}
        headers = {"X-RapidAPI-Key": config('FOOTBALL_API_KEY'), "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"}
        response = requests.request("GET", url, headers=headers, params=querystring)
        return response

    def format_date(self, response):
        #splits date and time string in to two separate strings, one with just the date in, one with time
        date_and_time = (response.json())['response'][0]['fixture']['date']
        
        split_date = date_and_time.split('T')

        #gets time in format 00:00 hour:minutes but in 24hr format
        time = (split_date[1])[:5]
        date = split_date[0]

        #turns time to 12hr format
        format_time_24hr = datetime.strptime(time, '%H:%M')
        format_time_12hr = format_time_24hr.strftime('%I:%M %p')

        #change date to uk format (DD/MM/YY)
        format_date_with_datetime = (datetime.strptime(date, '%Y-%m-%d'))
        format_date_to_uk_format = format_date_with_datetime.strftime('%a-%d-%B')

        format_date_and_time = format_date_to_uk_format + ' @ ' + format_time_12hr
        return format_date_and_time

    def extract_football_data(self, response, format_date_and_time):
       
        league = (response.json())['response'][0]['league']['name']

        home_team_name = (response.json())['response'][0]['teams']['home']['name']
        home_team_logo = (response.json())['response'][0]['teams']['home']['logo']

        away_team_name = (response.json())['response'][0]['teams']['away']['name']
        away_team_logo = (response.json())['response'][0]['teams']['away']['logo']

        match_teams = home_team_name + ' vs ' + away_team_name
        date_and_league = format_date_and_time + ' for ' + league

        top_row_widget = pn.widgets.StaticText(value=match_teams, width=200, style={'color':'white', 'font-family': "monospace"})
        home_logo = pn.pane.PNG(home_team_logo, width=100)
        away_logo = pn.pane.PNG(away_team_logo, width=100)
        middle_row_widget = pn.Row(home_logo, away_logo)
        bottom_row_widget = pn.widgets.StaticText(value=date_and_league, width=200, style={'color':'white', 'font-family': "monospace"})

        man_united_widget = pn.Column(top_row_widget, middle_row_widget, bottom_row_widget)

        return man_united_widget

    def football_api_process(self):
        response = self.get_manutd_next_game_data()
        format_date_and_time = self.format_date(response)
        football_widget = self.extract_football_data(response, format_date_and_time)
        return football_widget

category_graph = (MainGraphs()).make_category_graph()
detailed_graph = (MainGraphs()).make_detailed_graph()
heat_map = (MainGraphs()).make_heatmap()
box_plot = (MainGraphs()).make_box_plot()

top = pn.Row(category_graph, detailed_graph)
bottom = pn.Row(heat_map, box_plot)

main_items = pn.Column(top, bottom)

quote = (SideBar()).get_quote()
nasa_image = (SideBar()).nasa_image_of_the_day()
next_united_game = (SideBar()).football_api_process()

sidebar_items = pn.Column(quote, nasa_image, next_united_game)

template = pn.template.FastListTemplate(
        title ="Gabriella's Dashboard",
        sidebar = sidebar_items,
        main = main_items,
        theme='dark',
        theme_toggle=False,
        accent='hotpink',
        sidebar_width = 215
    )

template.servable()